In [1]:
import requests
import psycopg2
import pandas as pd
import numpy as np
import warnings

In [9]:
# Get data into a dataframe

api_url = "https://api.coingecko.com/api/v3/exchange_rates"
response = requests.get(api_url)

data = response.json()
rates = data['rates']

tempdf = pd.DataFrame.from_dict(rates)
df = tempdf.T

print(df)

                            name  unit        value       type
btc                      Bitcoin   BTC          1.0     crypto
eth                        Ether   ETH       13.585     crypto
ltc                     Litecoin   LTC      217.204     crypto
bch                 Bitcoin Cash   BCH      153.448     crypto
bnb                 Binance Coin   BNB       58.848     crypto
...                          ...   ...          ...        ...
xdr   IMF Special Drawing Rights   XDR    12761.096       fiat
xag          Silver - Troy Ounce   XAG      764.178  commodity
xau            Gold - Troy Ounce   XAU        9.578  commodity
bits                        Bits  μBTC    1000000.0     crypto
sats                     Satoshi  sats  100000000.0     crypto

[61 rows x 4 columns]


In [16]:
# Connect to RDS Postgres database
conn = psycopg2.connect(
    database="crypto",
    user="jkamino",
    password="Tacoman123",
    host="data-5330-1.ckpcm5tpab0e.us-west-1.rds.amazonaws.com",
    port="5432")

cursor = conn.cursor() 

In [4]:
# Create Table if it doesn't exist
query = """
CREATE TABLE IF NOT EXISTS exchange_rates (
  exchange_id serial PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  unit VARCHAR(5) NOT NULL,
  value FLOAT, 
  type VARCHAR(20), 
  date TIMESTAMP  
);
"""

cursor.execute(query)
conn.commit()

In [17]:
# Loop through all items in df, ingest
for x in df.index:
    cursor.execute(f"""
                    INSERT INTO exchange_rates ( name, unit, value, type, date)
                    VALUES ('{df.loc[x]["name"]}', '{df.loc[x]["unit"]}', '{df.loc[x]["value"]}', '{df.loc[x]["type"]}', current_timestamp)
                    """)

    conn.commit()
    
    name = df.loc[x]["name"]
    
    print(f"{name} ingested")
    

Bitcoin ingested
Ether ingested
Litecoin ingested
Bitcoin Cash ingested
Binance Coin ingested
EOS ingested
XRP ingested
Lumens ingested
Chainlink ingested
Polkadot ingested
Yearn.finance ingested
US Dollar ingested
United Arab Emirates Dirham ingested
Argentine Peso ingested
Australian Dollar ingested
Bangladeshi Taka ingested
Bahraini Dinar ingested
Bermudian Dollar ingested
Brazil Real ingested
Canadian Dollar ingested
Swiss Franc ingested
Chilean Peso ingested
Chinese Yuan ingested
Czech Koruna ingested
Danish Krone ingested
Euro ingested
British Pound Sterling ingested
Hong Kong Dollar ingested
Hungarian Forint ingested
Indonesian Rupiah ingested
Israeli New Shekel ingested
Indian Rupee ingested
Japanese Yen ingested
South Korean Won ingested
Kuwaiti Dinar ingested
Sri Lankan Rupee ingested
Burmese Kyat ingested
Mexican Peso ingested
Malaysian Ringgit ingested
Nigerian Naira ingested
Norwegian Krone ingested
New Zealand Dollar ingested
Philippine Peso ingested
Pakistani Rupee inges

In [20]:
# Lets see if this worked
query = (f"""
SELECT *
FROM exchange_rates
""")

df_sql_table = pd.read_sql(query, conn,index_col='exchange_id')

print(df_sql_table)

                                   name  unit         value       type  \
exchange_id                                                              
1                               Bitcoin   BTC  1.000000e+00     crypto   
2                                 Ether   ETH  1.358500e+01     crypto   
3                              Litecoin   LTC  2.172040e+02     crypto   
4                          Bitcoin Cash   BCH  1.534480e+02     crypto   
5                          Binance Coin   BNB  5.884800e+01     crypto   
...                                 ...   ...           ...        ...   
57           IMF Special Drawing Rights   XDR  1.276110e+04       fiat   
58                  Silver - Troy Ounce   XAG  7.641780e+02  commodity   
59                    Gold - Troy Ounce   XAU  9.578000e+00  commodity   
60                                 Bits  μBTC  1.000000e+06     crypto   
61                              Satoshi  sats  1.000000e+08     crypto   

                                  dat

C:\Users\JKami\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
